In [21]:
import pandas as pd

## Model Training

In [22]:
df = pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [23]:
X = df.drop(labels=['price'],axis=1)
y = df[['price']]

In [24]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [25]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [26]:
cut_categories = ["Fair",'Good',"Very Good","Premium",'Ideal']
color_categories = ["D","E","F","G","H","I","J"]
clarity_categories = ["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]

In [27]:
from sklearn.impute import SimpleImputer  ## Handle Missing Value
from sklearn.preprocessing import StandardScaler ## Handling Feature Scaling 
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [28]:
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

## Train Test Split

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [31]:
X_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.887629,2.626061,-2.888129,0.400868,2.233112,2.216066,1.856561,-0.130933,1.525655,-1.314696
1,-1.057304,-0.845291,0.164716,0.922458,-0.915966,-0.908068,-0.890852,-0.130933,-0.937159,-0.648656
2,-0.002860,-0.845291,-1.500472,1.965640,-0.843987,-0.899013,-0.963153,-0.130933,-0.321455,-0.648656
3,0.981911,-0.694363,-0.667878,-0.642314,-0.637048,-0.636405,-0.673951,0.874463,-0.937159,-1.314696
4,0.030009,1.548002,-0.482857,1.444049,1.477333,1.455407,1.393839,-0.130933,1.525655,0.683424


In [32]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [33]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [34]:
regression.coef_

array([[ 1.51837147e+00,  6.43264486e+03, -1.33123405e+02,
        -7.03718413e+01, -1.71320654e+03, -4.90479503e+02,
        -6.80606802e+01,  6.83656655e+01, -4.64256501e+02,
         6.51945495e+02]])

In [35]:
regression.coef_

array([[ 1.51837147e+00,  6.43264486e+03, -1.33123405e+02,
        -7.03718413e+01, -1.71320654e+03, -4.90479503e+02,
        -6.80606802e+01,  6.83656655e+01, -4.64256501e+02,
         6.51945495e+02]])

In [47]:
import numpy as np
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)

    return mae,rmse,r2_square

In [48]:
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    #Make_Predictions

    y_pred = model.predict(X_test)

    mae,rmse,r2_square = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Training Performance")
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("r2_score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1014.2778153382557
MAE: 674.734815118876
r2_score 93.63889934551443


Lasso
Model Training Performance
RMSE: 1014.3372093373042
MAE: 675.8980580607733
r2_score 93.63815433805705


Ridge
Model Training Performance
RMSE: 1014.2820747091499
MAE: 674.7683132311195
r2_score 93.63884591963156


ElasticNet
Model Training Performance
RMSE: 1534.1040071498533
MAE: 1061.324860120264
r2_score 85.44782116433484


